# Abstractive Text Summarization using Deep Learning

## Data Preprocessing and Cleaning

In [ ]:
from tqdm import tqdm
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

import re

In [ ]:
data = pd.read_csv('Data/amazon-fine-food-reviews.csv')
data.drop_duplicates(inplace=True) # dropping duplicates
data.dropna(axis=0, inplace = True) # dropping empty rows
data.info()

In [ ]:
data = data[:100000]

In [ ]:
data = data.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Score','Time'], 1)

In [ ]:
# function to carry out contraction expansions
def cleaner(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"y\'all", "you all", phrase)
    return phrase

In [ ]:
def clean(column_name):
    preprocessed_text = []
    for sentence in tqdm(data[column_name].values):
        try:
            sentence = sentence.lower() # converting all letters to lowercase
            sentence = re.sub(r"http\S+", "", sentence) # removing urls
            sentence = BeautifulSoup(sentence, 'lxml').get_text() # removing any html tags
            sentence = re.sub(r'\([^)]*\)', '', sentence) # removing any text inside parantheses
            sentence = cleaner(sentence) # contraction expansions
            sentence = re.sub(r"'s\b","", sentence) # removing apostrophes
            sentence = re.sub("\S*\d\S*", "", sentence).strip() # whitespace
            sentence = re.sub('[^A-Za-z]+', ' ', sentence) # removing special characters, punctuation marks
            preprocessed_text.append(sentence.strip())
        except:
            preprocessed_text.append('')
    return preprocessed_text

In [ ]:
data['Cleaned Summary'] = clean('Summary')
print("Summaries have been cleaned.")

In [ ]:
data['Cleaned Text'] = clean('Text')
print("Texts have been cleaned.")

In [ ]:
data.to_csv('Data/Cleaned Data.csv', index = False) # exporting as CSV for external use

In [ ]:
data = pd.read_csv('Data/Cleaned Data.csv')
data = data.replace(np.nan, '', regex = True)
data.info()

In [ ]:
for i in range(5):
    print("Review:",data['Cleaned Text'][i])
    print("Summary:",data['Cleaned Summary'][i])
    print("\n")

In [ ]:
count = 0
for i in data['Cleaned Summary']:
    if(len(i.split()) <= 10):
        count = count + 1
print(count / len(data['Cleaned Summary']))

In [ ]:
max_summary_len = 10
max_text_len = 50

In [ ]:
cleaned_text =np.array(data['Cleaned Text'])
cleaned_summary=np.array(data['Cleaned Summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
data=pd.DataFrame({'text':short_text,'summary':short_summary})

In [ ]:
data['summary'] = data['summary'].apply(lambda x : 'START '+ x + ' END')

In [ ]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(data['Cleaned Text'], data['Cleaned Summary'], test_size=0.2, random_state=0, shuffle=True) 

## Preparing Tokenizers

In [ ]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

### Text Tokenizer

In [ ]:
# initializing a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

# any word whose word count falls below 4 would be considered a rare word
threshold = 4
count = 0
total_count = 0
frequency = 0
total_frequency = 0

for key, value in x_tokenizer.word_counts.items():
    total_count = total_count + 1
    total_frequency = total_frequency + value
    if(value < threshold):
        count = count + 1
        frequency = frequency + value
    
print("Total % of rare words in vocabulary: ", (count / total_count) * 100)
print("Total coverage of rare words in vocabulary: ", (frequency / total_frequency) * 100)

# preparing a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words = total_count - count) 
x_tokenizer.fit_on_texts(list(x_tr))

# converting text sequences into integer sequences
x_tr_seq = x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq = x_tokenizer.texts_to_sequences(x_val)

# padding zero upto maximum length
x_tr = pad_sequences(x_tr_seq,  maxlen = max_text_len, padding = 'post')
x_val = pad_sequences(x_val_seq, maxlen = max_text_len, padding = 'post')

#size of vocabulary ( +1 for padding token)
x_voc = x_tokenizer.num_words + 1
print("Size of reviews vocabulary: ", x_voc)

### Summary Tokenizer

In [ ]:
# initialzing a tokenizer for summaries on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

# any word whose word count falls below 4 would be considered a rare word
thresh = 4
count = 0
total_count = 0
frequency = 0
total_frequency = 0

for key, value in y_tokenizer.word_counts.items():
    total_count = total_count + 1
    total_frequency = total_frequency + value
    if(value < thresh):
        count = count + 1
        frequency = frequency + value
    
print("% of rare words in vocabulary:", (count / total_count) * 100)
print("Total Coverage of rare words:", (frequency / total_frequency) * 100)

# preparing a tokenizer for summaries on training data
y_tokenizer = Tokenizer(num_words = total_count-count) 
y_tokenizer.fit_on_texts(list(y_tr))

# converting text sequences into integer sequences
y_tr_seq = y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq = y_tokenizer.texts_to_sequences(y_val) 

# padding zero upto maximum length
y_tr = pad_sequences(y_tr_seq, maxlen = max_summary_len, padding = 'post')
y_val = pad_sequences(y_val_seq, maxlen = max_summary_len, padding = 'post')

# size of vocabulary
y_voc = y_tokenizer.num_words + 1
print("Size of summary vocabulary: ", y_voc)

In [ ]:
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)
y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)
y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

## Model Building

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings

In [ ]:
from attention import AttentionLayer

In [ ]:
from keras import backend as K 
K.clear_session()

latent_dim = 300
embedding_dim = 100

# Encoder
encoder_inputs = Input(shape = (max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim, trainable = True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim, return_sequences = True, return_state = True, dropout = 0.4, recurrent_dropout = 0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim, return_sequences = True, return_state = True, dropout = 0.4, recurrent_dropout = 0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3 = LSTM(latent_dim, return_sequences = True, return_state = True, dropout = 0.4, recurrent_dropout = 0.4)
encoder_outputs, state_h, state_c = encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape = (None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim, trainable = True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences = True, return_state = True, dropout = 0.4, recurrent_dropout = 0.2)
decoder_outputs, decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb, initial_state = [state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name = 'attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis= -1, name = 'concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'sparse_categorical_crossentropy')

In [ ]:
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 2)

In [ ]:
history = model.fit([x_tr, y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)[:, 1:], epochs = 25, callbacks = [es], batch_size = 512, validation_data = ([x_val, y_val[:, :-1]], y_val.reshape(y_val.shape[0], y_val.shape[1], 1)[:, 1:]))

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
reverse_target_word_index = y_tokenizer.index_word
reverse_source_word_index = x_tokenizer.index_word
target_word_index = y_tokenizer.word_index

## Inference

In [ ]:
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs = encoder_inputs, outputs = [encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_hidden_state_input = Input(shape = (max_text_len, latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2 = dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = [decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis = -1, name = 'concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c], [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['_START_']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        if(sampled_token != '_END_'):
            decoded_sentence += ' '+sampled_token
        # Exit condition: either hit max length or find stop word.
        if (sampled_token == '_END_'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        # Update internal states
        e_h, e_c = h, c
    return decoded_sentence

In [ ]:
def seq2summary(input_seq):
    newString = ''
    for i in input_seq:
        if((i != 0 and i != target_word_index['_START_']) and i != target_word_index['_END_']):
            newString = newString + reverse_target_word_index[i] + ' '
    return newString

def seq2text(input_seq):
    newString = ''
    for i in input_seq:
        if(i != 0):
            newString = newString + reverse_source_word_index[i] + ' '
    return newString

In [ ]:
for i in range(0, 100):
    print("Review:", seq2text(x_tr[i]))
    print("Original summary:", seq2summary(y_tr[i]))
    print("Predicted summary:", decode_sequence(x_val[i].reshape(1, max_text_len)))
    print("\n")

In [ ]:
model.save("Models/Text Summarization")

In [ ]:
model.save('model.h5')